In [ ]:
!pip install transformers
!git clone https://github.com/aub-mind/arabert
!pip install pyarabic
!pip install farasapy

In [13]:
from transformers import AutoTokenizer, AutoModel
from arabert.preprocess_arabert import never_split_tokens, preprocess
from farasa.segmenter import FarasaSegmenter

import torch

arabert_tokenizer = AutoTokenizer.from_pretrained(
    "aubmindlab/bert-base-arabert",
    do_lower_case=False,
    do_basic_tokenize=True,
    never_split=never_split_tokens)
arabert_model = AutoModel.from_pretrained("aubmindlab/bert-base-arabert") #you can replace the path here with the folder containing the the pytorch model

farasa_segmenter = FarasaSegmenter(interactive=True)

NameError: name 'transformers' is not defined

In [2]:
text= "الجو جميل اليوم"
text_preprocessed = preprocess( text,
                                do_farasa_tokenization = True,
                                farasa = farasa_segmenter,
                                use_farasapy = True) # if you want to use AraBERT v0.1 do_farasa_tokenization = False
print(text)
print("---------------------")
print(text_preprocessed)

الجو جميل اليوم
---------------------
ال+ جو جميل ال+ يوم


In [3]:
arabert_input = arabert_tokenizer.encode(text_preprocessed,add_special_tokens=True)
print(arabert_input)
print(arabert_tokenizer.convert_ids_to_tokens(arabert_input))
# you should ignore the fisrt and the last embeddings

[29756, 3000, 516, 15724, 3000, 7447, 29758]
['[CLS]', 'ال+', 'جو', 'جميل', 'ال+', 'يوم', '[SEP]']


In [4]:
tensor_input_ids = torch.tensor(arabert_input).unsqueeze(0)
print(tensor_input_ids)

tensor([[29756,  3000,   516, 15724,  3000,  7447, 29758]])


In [5]:
output = arabert_model(tensor_input_ids)


IndexError: tuple index out of range

In [9]:
output[0].shape

torch.Size([1, 7, 768])

In [ ]:
output[0].shape # batch_size x seq_len x emb_dim

In [ ]:
embeddings = output[0][0][1:-1]

In [ ]:
print(embeddings.shape)
print(embeddings)

In [ ]:
import pandas as pd

header_list = ["word", "body", "def",'quran_ex','example']


df = pd.read_csv(r"C:\Users\me250041\OneDrive - Teradata\Desktop\master\scrapping\data-factory\new_scrapping_all_out.txt",delimiter='$',encoding='utf-8',names=header_list)


df[['word','def']]

In [ ]:
def farasa_segmenting(text):
    text_preprocessed = preprocess( text,
                                do_farasa_tokenization = True,
                                farasa = farasa_segmenter,
                                use_farasapy = True)
    return text_preprocessed


In [ ]:
df['def_tokenized']=df.apply(lambda row: farasa_segmenting(row['def']),axis=1)

In [ ]:
df

In [ ]:
output[1].shape


In [ ]:
def segmenting_examples(quran_ex,ex):
    if quran_ex.strip() != "":
        return farasa_segmenting(quran_ex)
    elif ex.strip() != '':
        return farasa_segmenting(ex)
    else:
        return None

In [ ]:
def choose_example(quran_ex,ex):
    if quran_ex.strip() != "":
        return quran_ex
    elif ex.strip() != '':
        return ex
    else:
        return None

In [ ]:
df['example_tokenized']=df.apply(lambda row: segmenting_examples(row['quran_ex'],row['example']),axis=1)

In [ ]:
df['choosed_Example']=df.apply(lambda row: choose_example(row['quran_ex'],row['example']),axis=1)

In [ ]:
import re
from farasa.stemmer import FarasaStemmer
FarasaStemmer = FarasaStemmer(interactive=True)


In [ ]:
def segmenting_word(word):
   
    seg = farasa_segmenting(word)
    seg_list= re.split('\s+',seg)
    for s in seg_list:
        if '+' not in s:
            return s
    
    max_i= -1
    max_l= -1
    i = 0
    for s in seg_list:
        if len(s)>max_l:
            max_l = len(s)
            max_i = i
        i += 1
    return seg_list[max_i]

In [ ]:
df['word_stemed']=df.apply(lambda row: FarasaStemmer.stem(row['word']),axis=1)

In [ ]:
df['word_segmented']=df.apply(lambda row:segmenting_word(row['word']),axis=1)

In [ ]:
df

In [ ]:
def example_encoding(example_tokenized,example,word,segmented_word):
    word = word.strip()
    example_tokenized = str(example_tokenized)
    word = str(word)
    example_tokenized = re.sub('\+',' ',example_tokenized)
    example_tokenized = re.sub('\s+',' ',example_tokenized)
    example_tokenized_list = re.split('\s+',example_tokenized)
    x = -1
    if word not in example_tokenized_list:
      

        if segmented_word in example_tokenized_list:
            x = example_tokenized_list.index(segmented_word)
      
        elif word[-1] in ['ه','ة']:
            sub_word = word[:-1]
            if sub_word in example_tokenized_list:
                x = example_tokenized_list.index(sub_word)
        elif segmented_word[-1] in ['ه','ة']:
            sub_word = segmented_word[:-1]
            if sub_word in example_tokenized_list:
                x = example_tokenized_list.index(sub_word)
        else:
            for i in range(len(example_tokenized_list)):
                if word in example_tokenized_list[i]:
                    x = i
                    break
        
        if x ==-1:
            return None
                    
        
    else:
        x = example_tokenized_list.index(word)
    arabert_input = arabert_tokenizer.encode(example_tokenized,add_special_tokens=True)
#     print(arabert_input)
#     print(arabert_tokenizer.convert_ids_to_tokens(arabert_input))
    tensor_input_ids = torch.tensor(arabert_input).unsqueeze(0)
    output = arabert_model(tensor_input_ids)
    
    embeddings = output[0][0][1:-1]
    
    return embeddings[x].tolist()
# you should ignore the fisrt and the last embeddings
    

In [ ]:
output[0].shape

In [ ]:
output[1].shape

In [ ]:
df['context_word_embeddings'] = df.apply(lambda row: example_encoding(row['example_tokenized'],row['choosed_Example'],row['word'],row['word_segmented']),axis=1)

In [ ]:
def def_encoding(defnition):
    
    defnition = str(defnition)
    definition_splited = re.split('\s+',defnition)
    print(len(definition_splited))
    if len(definition_splited)>450:
        print(defnition)
        return None
    defnition = re.sub('\s+',' ',defnition)
#     print('definition: ',defnition)
    arabert_input = arabert_tokenizer.encode(defnition,add_special_tokens=True)
#     print(arabert_input)
#     print(arabert_tokenizer.convert_ids_to_tokens(arabert_input))
    tensor_input_ids = torch.tensor(arabert_input).unsqueeze(0)
    output = arabert_model(tensor_input_ids)
    
    embeddings = output[1]
    
    return embeddings.tolist()

In [ ]:
df

In [ ]:
df['def_embeddings']=df.apply(lambda row: def_encoding(row['def_tokenized']),axis=1)

In [ ]:
exists =df.dropna(subset=['context_word_embeddings'])


In [ ]:
exists.count()

In [ ]:
exists

In [ ]:
not_exist = df[ df['example_tokenized'].isnull()]

In [ ]:
not_exist

In [ ]:
exists.to_csv('./exists.csv',encoding='utf-8',index=False)

In [ ]:
df.to_csv('./df.csv',encoding='utf-8',index=False)

In [ ]:
df2 ="pd.read_csv('./df.csv')"

In [1]:
import pandas as pd

In [2]:
exists = pd.read_csv('./exists.csv',encoding='utf-8')

In [3]:
exists

,word,body,def,quran_ex,example,def_tokenized,example_tokenized,context_word_embeddings,choosed_Example,word_segmented,word_stemed,def_embeddings
0,عرض,عرض / عرض لـ يعرض عرضا وعروضا فهو عارض وال...,/ عرض لـ يعرض عرضا وعروضا فهو عارض والمفع...,وعرضنا جهنم يومئذ للكافرين عرضا,عرض خطة بحثه - {وعلم ءادم الأسماء كلها ثم عرض...,/ عرض ل+ يعرض عرض +ا و+ عروض +ا ف+ هو عارض و+ ...,و+ عرض +نا جهنم يومئذ ل+ ال+ كافر +ين عرض +ا,"[0.08959540724754333, 0.2315646857023239, 0.19...",وعرضنا جهنم يومئذ للكافرين عرضا,عرض,عرض,"[[-0.9497016072273254, 0.059308554977178574, -..."
1,عرض,عرض / عرض بـ / عرض لـ يعرض تعريضا فهو معرض ...,/ عرض بـ / عرض لـ يعرض تعريضا فهو معرض وا...,ولا جناح عليكم فيما عرضتم به من خطبة النساء,عرض الطريق جعلها عريضة واسعة,/ عرض ب+ / عرض ل+ يعرض تعريض +ا ف+ هو معرض و+ ...,و+ لا جناح علي +كم فيما عرضتم ب+ +ه من خطب +ة ...,"[-0.13121910393238068, 0.08852916955947876, 0....",ولا جناح عليكم فيما عرضتم به من خطبة النساء,عرض,عرض,"[[-0.8836850523948669, 0.05310528352856636, -0..."
2,عرض,عرض يعرض عرضا وعرضا وعراضة فهو عريض :- عر...,يعرض عرضا وعرضا وعراضة فهو عريض :- :-عريض ...,,عرض الثوب تباعدت حاشيتاه واتسع عرضه خلاف طال,يعرض عرض +ا و+ عرض +ا و+ عراض +ة ف+ هو عريض :-...,عرض ال+ ثوب تباعد +ت حاشيتاه و+ اتسع عرض +ه خل...,"[0.25407180190086365, 0.2322186827659607, 0.36...",عرض الثوب تباعدت حاشيتاه واتسع عرضه خلاف طال,عرض,عرض,"[[0.046237848699092865, 0.024290665984153748, ..."
3,عرض,عرض :- جمع أعراض: 1 - اسم لما لا دوام له :-ه...,:- جمع أعراض: 1 - اسم لما لا دوام له :-هذا ...,لو كان عرضا قريبا وسفرا قاصدا لاتبعوك,{تريدون عرض الدنيا والله يريد الآخرة} .3 - نفع,:- جمع أعراض : 1 - اسم ل+ ما لا دوام ل+ +ه :- ...,لو كان عرض +ا قريب +ا و+ سفر +ا قاصد +ا لاتبعوك,"[-0.2285122275352478, -0.035964034497737885, 0...",لو كان عرضا قريبا وسفرا قاصدا لاتبعوك,عرض,عرض,"[[-0.8642708659172058, 0.017605777829885483, 0..."
4,عرض,عرض :- جمع أعراض: ما يمدح ويذم من الإنسان في...,:- جمع أعراض: ما يمدح ويذم من الإنسان في نف...,,طعن في عرض فلان - لا تجرح أعراض الناس,:- جمع أعراض : ما يمدح و+ يذم من ال+ إنسان في ...,طعن في عرض فلان - لا تجرح أعراض ال+ ناس,"[-0.23766028881072998, 0.37875208258628845, -0...",طعن في عرض فلان - لا تجرح أعراض الناس,عرض,عرض,"[[-0.7836717963218689, 0.03240634873509407, 0...."
...,...,...,...,...,...,...,...,...,...,...,...,...
10102,عيسى,عيسى :- اسم السيد المسيح عليه السلام وهو اسم...,:- اسم السيد المسيح عليه السلام وهو اسم عبر...,وقفينا بعيسى ابن مريم وءاتيناه الإنجيل,,:- اسم ال+ سيد ال+ مسيح علي +ه ال+ سلام و+ هو ...,وقفي +نا ب+ عيسى ابن مريم وءاتيناه ال+ إنجيل,"[0.28636354207992554, 0.37235116958618164, 0.2...",وقفينا بعيسى ابن مريم وءاتيناه الإنجيل,عيسى,عيسى,"[[0.9598995447158813, 0.08008348196744919, 0.0..."
10103,أخف,أخف يخف أخفف / أخف إخفافا فهو مخف والمفعو...,يخف أخفف / أخف إخفافا فهو مخف والمفعول م...,,أخف الرجل قلل متاعه في السفر,يخف أخفف / أخف إخفاف +ا ف+ هو مخف و+ ال+ مفعول...,أخف ال+ رجل قلل متاع +ه في ال+ سفر,"[0.013894468545913696, 0.05375600606203079, 0....",أخف الرجل قلل متاعه في السفر,أخف,أخف,"[[0.16920645534992218, 0.09069421887397766, 0...."
10104,كمبيالة,كمبيالة :- (التجارة) وثيقة يتعهد فيها المدين...,:- (التجارة) وثيقة يتعهد فيها المدين بأن يد...,,تأخر عن دفع الكمبيالة,:- ( ال+ تجار +ة ) وثيق +ة يتعهد في +ها ال+ مد...,تأخر عن دفع ال+ كمبيال +ة,"[-0.5766795873641968, -0.2895876169204712, 0.6...",تأخر عن دفع الكمبيالة,كمبيال,كمبيال,"[[-0.9263273477554321, 0.08698716014623642, 0...."
10105,صك,صك صككت يصك اصكك / صك صكا فهو صاك والمفع...,صككت يصك اصكك / صك صكا فهو صاك والمفعول...,فأقبلت امرأته في صرة فصكت وجهها,صك الشخص دفعه بقوة,صكك +ت يصك اصكك / صك صك +ا ف+ هو صاك و+ ال+ مف...,ف+ أقبل +ت امرأ +ت +ه في صر +ة ف+ صك +ت وجه +ها,"[0.002387251704931259, -0.4425205886363983, 0....",فأقبلت امرأته في صرة فصكت وجهها,صك,صك,"[[-0.9401823878288269, 0.06284652650356293, 0...."


In [4]:
exists['context_word_embeddings'][0]


'[0.08959540724754333, 0.2315646857023239, 0.1970081329345703, 0.023173851892352104, -0.08430591225624084, -0.7117202281951904, -0.4230063557624817, -0.4335278570652008, 0.04300980269908905, 0.24796119332313538, 0.08111585676670074, -0.12286245077848434, 0.019916865974664688, -0.2145615518093109, -0.30962371826171875, 0.2771870493888855, 0.845855712890625, -0.16842609643936157, -0.06432172656059265, -0.05510283634066582, -0.08768151700496674, -0.02053827978670597, -0.42498520016670227, -0.3374709486961365, -0.6844677329063416, -0.19126152992248535, 0.3672560751438141, -0.8060681223869324, 0.28782278299331665, 0.16762182116508484, 0.2361242026090622, 0.28043559193611145, -1.062125325202942, -0.2066148966550827, -0.4131197929382324, 0.5782737135887146, 0.621004045009613, 0.5171436667442322, 0.37186315655708313, 0.42654547095298767, -0.589360773563385, -0.44860243797302246, -0.13196268677711487, -0.760196328163147, -0.13935643434524536, 0.01865604892373085, -0.07312328368425369, -0.635566

In [ ]:
exists['context_word_embeddings'][0]

In [5]:
import re

In [6]:
def convert_to_list(string_list):
    string_list = str(string_list)
    string_list = re.sub(r'[\]\[]','',string_list)
    string_list = re.split(',\s+',string_list)
    float_list = [float(i) for i in string_list]
    return float_list

In [7]:
exists['def_embeddings'] = exists.apply(lambda row: convert_to_list(row['def_embeddings']),axis=1)

In [8]:
exists['context_word_embeddings'] = exists.apply(lambda row: convert_to_list(row['context_word_embeddings']),axis=1)

In [ ]:
exists['def_embeddings'][0]

In [ ]:
len(exists)

In [9]:
exists = exists.dropna()

In [10]:
def removeNA(list_n):
    for item in list_n:
        if type(item) != float or len(list_n)<700:
            return None
    return list_n

In [11]:
exists['def_embeddings'] = exists.apply(lambda row : removeNA(row['def_embeddings']),axis=1)

In [12]:
import numpy as np

In [ ]:
exists['def_embeddings'] = exists.apply(lambda row : np.array(row['def_embeddings']),axis=1)

In [ ]:
exists['context_word_embeddings'] = exists.apply(lambda row : np.array(row['context_word_embeddings']),axis=1)

In [ ]:
len(exists)

In [ ]:
exists = exists.dropna()
exists

In [13]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
# df_lable = pd.DataFrame(list(exists['def_embeddings']))

In [ ]:
# len(df_lable)

In [ ]:
# len(df_lable.dropna())

In [ ]:
# df_Dataset = pd.DataFrame(list(exists['context_word_embeddings']))

In [ ]:
# new_df =  pd.concat([df_Dataset, df_lable], axis=1, sort=False)


In [ ]:
# new_df = new_df.dropna()

In [ ]:
# new_df.columns

In [14]:
import numpy as np
msk = np.random.rand(len(exists)) < 0.8

In [15]:
train = exists[msk]

test = exists[~msk]

In [ ]:
train

In [ ]:
test

In [16]:
df_train_lable = train['context_word_embeddings']

In [17]:
df_test_lable = test['context_word_embeddings']

In [18]:
df_train_feat = train['def_embeddings']

In [19]:
df_test_feat = test['def_embeddings']

In [28]:
type(df_test_feat)

pandas.core.series.Series

In [18]:
df = pd.DataFrame(list(df_train_feat))

In [25]:
df_test = pd.DataFrame(list(df_test_feat))

In [20]:
df2 = pd.DataFrame(list(df_train_lable))

In [26]:
df2_test = pd.DataFrame(list(df_test_lable))

In [21]:
train_dataset = tf.data.Dataset.from_tensor_slices((df.values, df2.values))

In [27]:
test_dataset = tf.data.Dataset.from_tensor_slices((df_test.values, df2_test.values))

In [29]:
train_dataset = train_dataset.shuffle(len(exists)).batch(1)

In [32]:
len(train_dataset)

7967

In [31]:
for feat, targ in train_dataset.take(1):
  print ('Features: {}, Target: {}'.format(feat, targ))
#   print(feat.shape)
#   print(targ.shape)  

Features: [[-7.13348091e-01  1.69974387e-01 -1.06794365e-01  5.92167377e-02
   3.32967043e-02 -1.48030519e-01 -5.76183975e-01 -8.87805074e-02
   8.22577655e-01 -2.29942635e-01 -9.14840877e-01 -2.18333509e-02
   6.24106452e-02 -7.51832843e-01 -3.23265076e-01 -1.50983796e-01
  -3.97569826e-03  9.87466872e-01  9.99492407e-01 -2.19634533e-01
  -2.21089214e-01  9.51512083e-02  6.21492527e-02  9.85454768e-02
  -2.73770005e-01 -7.62048364e-01 -7.65762553e-02 -1.31656036e-01
   9.57463861e-01 -9.86426920e-02 -7.30633438e-02 -1.59533679e-01
  -1.22830078e-01 -6.64784610e-01 -4.94097322e-02  4.17975597e-02
  -2.83071101e-02  1.16813831e-01  9.94509876e-01  9.97694552e-01
   1.73136473e-01 -3.36099081e-02 -2.07418546e-01  3.01215142e-01
   9.90723014e-01 -2.99443007e-01 -9.84479368e-01  4.11359929e-02
  -3.50806236e-01  2.00598910e-02 -5.66493750e-01  1.29915476e-01
  -7.11671892e-04 -1.43012434e-01 -2.99592942e-01  6.41603351e-01
   1.25105679e-01 -7.94131830e-02 -4.81505930e-01  9.94015634e-01


In [37]:
from tensorflow.keras import regularizers
opt = keras.optimizers.Adam(learning_rate=0.001)

def get_compiled_model():
  model = tf.keras.Sequential([
    tf.keras.layers.Dense(350, activation='relu',kernel_regularizer=regularizers.l2(l2=1e-4)),
    tf.keras.layers.Dense(200, activation='relu',kernel_regularizer=regularizers.l2(l2=1e-4)),
    tf.keras.layers.Dense(150, activation='relu',kernel_regularizer=regularizers.l2(l2=1e-4)),
    tf.keras.layers.Dense(768)
  ])

  model.compile(optimizer=opt,
                loss=tf.keras.losses.cosine_similarity,
                metrics='mse'
                )
  return model

In [38]:
model = get_compiled_model()
model.fit(train_dataset, epochs=15)

Epoch 1/15

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

7967/7967 [==============================] - 59s 7ms/step - loss: -0.7385 - mse: 46.5084
Epoch 2/15
7967/7967 [==============================] - 59s 7ms/step - loss: -0.7485 - mse: 0.4865
Epoch 3/15
7967/7967 [==============================] - 60s 7ms/step - loss: -0.7501 - mse: 0.4162
Epoch 4/15
7967/7967 [==============================] - 59s 7ms/step - loss: -0.7514 - mse: 0.3857
Epoch 5/15
7967/7967 [==============================] - 62s 8ms/step - loss: -0.7522 - mse: 0.4464
Epoch 6/15
7

In [39]:
model.evaluate(df_test.values,df2_test.values)

65/65 [==============================] - 0s 5ms/step - loss: -0.7579 - mse: 0.8006


[-0.757902979850769, 0.8006114959716797]

In [40]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 350)               269150    
_________________________________________________________________
dense_5 (Dense)              (None, 200)               70200     
_________________________________________________________________
dense_6 (Dense)              (None, 150)               30150     
_________________________________________________________________
dense_7 (Dense)              (None, 768)               115968    
Total params: 485,468
Trainable params: 485,468
Non-trainable params: 0
_________________________________________________________________


In [ ]:
import tensorflow as tf
def predict(py_list):
    return model.predict(tf.convert_to_tensor(py_list))
    

In [ ]:
model.predict(tf.reshape(tf.convert_to_tensor(exists['context_word_embeddings'][0]),(1,-1)))

In [41]:
exists['predictions'] = exists.apply(lambda row: model.predict(tf.reshape(tf.convert_to_tensor(row['context_word_embeddings']),(1,-1))),axis=1)

In [ ]:
exists['predictions'][0][0]

In [ ]:
# exists = exists.reset_index()

In [43]:
exists['ID']=exists.index+1

In [44]:
exists.columns

Index(['word', 'body', 'def', 'quran_ex', 'example', 'def_tokenized',
       'example_tokenized', 'context_word_embeddings', 'choosed_Example',
       'word_segmented', 'word_stemed', 'def_embeddings', 'predictions', 'ID'],
      dtype='object')

In [48]:
def preprocess(nd_arr):
    return list(nd_arr[0])

In [53]:
exists['predictions_preprocessed'] = exists.apply(lambda row :preprocess(row['predictions']),axis=1)

In [ ]:
exists['wordandID'] = exists.apply(lambda row : row['word'].strip()+str(row['ID'].strip()),axis=1)

In [81]:
exists[['wordandID','predictions_preprocessed']].to_csv('./prdictions_embeddings.csv',encoding='utf-8',index=False)

In [82]:
exists[['wordandID','def_embeddings']].to_csv('./def_embeddings.csv',encoding='utf-8',index=False)

In [ ]:
exists['predictions_preprocessed'] 

In [ ]:
exists_copy = pd.DataFrame.copy(exists)

In [ ]:
exists.to_csv('./exists_after_prediction',encoding='utf-8',index=False)

In [56]:
from sklearn import preprocessing

In [59]:
exists['norm_prerdictions'] = exists.apply(lambda row :preprocessing.normalize(row['predictions'],norm='l2'),axis=1)

In [63]:
exists['norm_def'] = exists.apply(lambda row :preprocessing.normalize([row['def_embeddings']],norm='l2'),axis=1)

In [67]:
type(exists['norm_prerdictions'][0])

numpy.ndarray

In [68]:
exists['vector_multi'] = exists.apply(lambda row : np.multiply(row['norm_def'],row['norm_prerdictions']),axis=1)

In [ ]:
exists['cosine']

In [ ]:
from scipy import spatial
def predictions_cosine_smilarity(prdiction,df):
    prediction = list(prdiction)
    prediction = prdiction[0]
    df['cos_similarity_temp'] = df.apply(lambda row :1 - spatial.distance.cosine(prediction,row['def_embeddings']),axis=1)
    ordered_df = df.sort_values('cos_similarity_temp',ascending=False)
    return (ordered_df["ID"][0], ordered_df["ID"][1], ordered_df["ID"][2])
exists["cos_similarity"] = exists.apply(lambda row: predictions_cosine_smilarity(row['predictions'],exists_copy),axis=1)

In [ ]:
exists

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer(r'C:\Users\me250041\OneDrive - Teradata\Desktop\master\bert-large-nli-stsb-mean-tokens')

In [ ]:
sentences = ['This framework generates embeddings for each input sentence',
    'Sentences are passed as a list of string.', 
    'The quick brown fox jumps over the lazy dog.']
sentence_embeddings = model.encode(sentences)

In [ ]:
sentence_embeddings
